In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import j1  # Bessel function of the first kind
import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
# Constants
f = 16  # Frequency in kHz
rho0 = 1
c = 1500
a = 1
r = 10

omega = 2 * np.pi * f * 1000
k = omega / c

theta = np.arange(-90, 90.1, 0.1)
u = k * a * np.sin(theta * np.pi / 180)

# Projection Intensity calculation
projInt = np.abs((rho0 * c * k * a * a / (2 * r)) * 
                 (2 * j1(u) / u)) ** 2

# Handle division by zero at theta = 0
ind = np.where(theta == 0)[0]
axisInt = np.abs(rho0 * c * k * a * a / (2 * r)) ** 2
projInt[ind] = axisInt

# Pressure nulls calculation
zerosBesselJ1 = np.array([3.83, 7.02, 10.17])
pressureNulls = np.arcsin(zerosBesselJ1 / (k * a)) * 180 / np.pi
thetaInd = np.where((theta > pressureNulls[0]) & (theta < pressureNulls[1]))[0]
SideLobeDB = np.max(10 * np.log10(projInt[thetaInd] / axisInt))
indSideLobe = np.argmax(10 * np.log10(projInt[thetaInd] / axisInt))
firstSideLobeAngle = theta[thetaInd[indSideLobe]]

# Function to update plot based on widget input
def update_plot(plotType, plotScale):
    plt.figure(1)
    plt.clf()
    
    if plotType == 'Cartesian':
        if plotScale == "Decibels":
            plt.plot(theta, 10 * np.log10(projInt / axisInt), linewidth=2)
            plt.ylim([-40, 0])
            plt.xlim([-90, 90])
            plt.xlabel('Angle (degrees)', fontsize=14)
            plt.ylabel('Beam Level (dB)', fontsize=14)
            plt.grid(True)
        else:
            plt.plot(theta, projInt / axisInt, linewidth=2)
            plt.ylim(10 ** (np.array([-40, 0]) / 10))
            plt.xlim([-90, 90])
            plt.xlabel('Angle (degrees)', fontsize=14)
            plt.ylabel('Beam Level', fontsize=14)
            plt.grid(True)
    
    elif plotType == 'Polar':
        if plotScale == "Decibels":
            plt.polar(theta * np.pi / 180, 10 * np.log10(projInt / axisInt), linewidth=2)
            ax = plt.gca()
            ax.set_ylim([-40, 0])
            ax.set_theta_zero_location('N')  # 'top'
            ax.set_thetalim([-90, 90])
            ax.set_title('Beam Pattern (dB)', fontsize=14)
        else:
            plt.polar(theta * np.pi / 180, projInt / axisInt, linewidth=2)
            ax = plt.gca()
            ax.set_ylim(10 ** (np.array([-40, 0]) / 10))
            ax.set_theta_zero_location('N')  # 'top'
            ax.set_thetalim([-90, 90])
            ax.set_title('Beam Pattern (Linear Scale)', fontsize=14)
    
    plt.show()

# Create widgets for plot type and scale
plot_type_widget = widgets.Dropdown(
    options=['Cartesian', 'Polar'],
    value='Polar',
    description='Plot Type:'
)

plot_scale_widget = widgets.Dropdown(
    options=['Decibels', 'Linear'],
    value='Decibels',
    description='Scale:'
)

# Use interact to link widgets and plotting function
interact(update_plot, plotType=plot_type_widget, plotScale=plot_scale_widget)
